In [1]:
import pandas as pd
from utils.gridsearch import gridsearch
from utils.read2df import read2df
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint

import backtrader as bt
from itertools import combinations

In [2]:
symbols = ['BTCUSDT', 'ETHUSDT']

freqs = {'1m': 1, '3m': 3, '5m': 5, '15m': 15, '30m': 30, '1h':60, '2h': 120}

# Download Data from [binance-public-data](https://github.com/binance/binance-public-data/tree/master/python)

Download BTCUSDT and ETHUSDT for all available history for intervals of 1m, 3m, 5m, 15m, 30m


In [113]:
%%capture
!python binance-public-data/python/download-kline.py -t spot -s BTCUSDT ETHUSDT -i 1m 3m 5m 15m 30m 1h 2h -skip-daily 1

# Read Downloaded Data

The downloaded data can be found in `binance-public-data/data/`

In [3]:
dfs = read2df(symbols, freqs)

In [4]:
# I want to operate on:

FREQUENCY = '2h'
f = list(freqs.keys()).index(FREQUENCY)
dfs[f]

,time,open,high,low,close,volume,tic,itvl,datetime
0,1502949599999,4261.48,4328.69,4261.32,4315.32,70.415925,BTCUSDT,2h,2017-08-17 05:59:59.999
1,1502949599999,301.13,303.28,298.00,303.10,503.341230,ETHUSDT,2h,2017-08-17 05:59:59.999
2,1502956799999,4330.29,4349.99,4287.41,4349.99,11.672940,BTCUSDT,2h,2017-08-17 07:59:59.999
3,1502956799999,302.40,307.96,301.90,307.96,1058.611820,ETHUSDT,2h,2017-08-17 07:59:59.999
4,1502963999999,4333.32,4445.78,4333.32,4444.00,11.736430,BTCUSDT,2h,2017-08-17 09:59:59.999
...,...,...,...,...,...,...,...,...,...
52769,1693511999999,1660.16,1664.63,1645.00,1657.12,33238.020600,ETHUSDT,2h,2023-08-31 19:59:59.999
52770,1693519199999,26162.79,26194.86,25655.01,26024.57,7002.831350,BTCUSDT,2h,2023-08-31 21:59:59.999
52771,1693519199999,1657.12,1657.36,1630.00,1647.68,46938.519300,ETHUSDT,2h,2023-08-31 21:59:59.999
52772,1693526399999,26024.57,26072.73,25912.01,25940.78,2197.697860,BTCUSDT,2h,2023-08-31 23:59:59.999


# Check Cointegration and Correlation

In [5]:
for comb in combinations(set(dfs[f]['tic']), 2):
    first_ele = dfs[f][dfs[f]['tic']==comb[0]]['close']
    second_ele = dfs[f][dfs[f]['tic']==comb[1]]['close']
    _, pvalue, _ = coint(first_ele, second_ele)
    if pvalue <= 0.05:
        print(f"{comb[0]} and {comb[1]} are cointegrated under {FREQUENCY} interval")
    else:
        print(f"{comb[0]} and {comb[1]} are not cointegrated under {FREQUENCY} interval")

ETHUSDT and BTCUSDT are not cointegrated under 2h interval


# Define Trading Strategy

In [6]:
class PairTrading(bt.Strategy):
    params = dict(
        OPEN_THRE=2,
        CLOS_THRE=0.1,
        period=30
    )

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status == order.Completed:
            if order.isbuy():
                print(f"Buy {order.data._name} @ price: {order.executed.price} for Qty: {order.executed.size}")
            else:
                print(f"Sell {order.data._name} @ price: {order.executed.price} for Qty: {order.executed.size}")

        elif order.status in [order.Expired, order.Canceled, order.Margin]:
            print('%s ,' % order.Status[order.status])
            pass

    def __init__(self):
        self.data0 = self.datas[0]
        self.data1 = self.datas[1]

        # Calculate zscore of the ratio
        self.transform = bt.indicators.OLS_TransformationN(
            self.data1, self.data0, period=self.p.period
        )
        self.zscore = self.transform.zscore

        # -1 for short data1/data0, 1 for long data1/data0, 0 for no position
        self.position_status = 0

    def next(self):
        # print(f'Right now the zscore is {self.transform.zscore[0]}, and the position is {self.position_status}')
        
        # Calculate the ratio between the 2 assets
        ratio = self.data1.close[0] / self.data0.close[0]

        if abs(self.zscore[0]) < self.p.CLOS_THRE and self.position_status != 0:
            print("------")
            print("close position")
            self.position_status = 0
            self.close(data=self.data0)
            self.close(data=self.data1)
    
        elif self.zscore[0] < -self.p.OPEN_THRE and self.position_status == 0:
            print("------")
            print(f"long {self.data0.alias} and short {self.data1.alias}")
            self.position_status = 1
            self.sell(data=self.data0, size=ratio)
            self.buy(data=self.data1, size=1)

        elif self.zscore[0] > self.p.OPEN_THRE and self.position_status == 0:
            print("------")
            print(f"long {self.data1.alias} and short {self.data0.alias}")
            self.position_status = -1

            self.sell(data=self.data0, size=ratio)
            self.buy(data=self.data1, size=1)

    def stop(self):
        print('==================================================')
        print('Starting Value - %.2f' % self.broker.startingcash)
        print('Ending   Value - %.2f' % self.broker.getvalue())
        print('==================================================')

# Execute the Strategy

Load the data

In [7]:
datafeeds_eth = []
datafeeds_btc = []

for idx, freq in enumerate(freqs):
    datafeeds_eth.append(
        bt.feeds.PandasData(
            dataname=dfs[idx][dfs[idx]['tic']=='ETHUSDT'],
            datetime='datetime',
            open='open',
            high='high',
            low='low',
            close='close',
            volume='volume',
            openinterest=None
        )
    )
    datafeeds_btc.append(
        bt.feeds.PandasData(
            dataname=dfs[idx][dfs[idx]['tic']=='BTCUSDT'],
            datetime='datetime',
            open='open',
            high='high',
            low='low',
            close='close',
            volume='volume',
            openinterest=None
        )
    )
    print(freq)

1m
3m
5m
15m
30m
1h
2h


The main strategy engine

In [11]:
datafeed0 = datafeeds_eth[f]
datafeed1 = datafeeds_btc[f]
datafeeds = [datafeed0, datafeed1]
param = {'OPEN_THRE':1, 'CLOS_THRE':0.1, 'period':30}

def cerebro_run(datafeeds, param):
    # Create a Cerebro instance and add the data feed
    cerebro = bt.Cerebro()
    cerebro.adddata(datafeeds[0], name='eth')
    cerebro.adddata(datafeeds[1], name='btc')

    # Set up other parameters for your backtest
    cerebro.broker.set_cash(100000)  # Set initial capital
    cerebro.broker.setcommission(commission=0.001)  # Set commission rate

    # cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='timereturns')
    cerebro.addanalyzer(bt.analyzers.Returns, _name='Returns')

    cerebro.addstrategy(PairTrading, **param)
    strats = cerebro.run()
    return strats

# cerebro_run(datafeeds, param)

# Grid Search the Strategy
Define scoring and param_grid

In [16]:
param_grid = {
    'OPEN_THRE':[1, 2], 
    'CLOS_THRE':[0.1], 
    'period':[30]
}

def scoring(strats):
    score = strats[0].analyzers.Returns.get_analysis()['rtot']
    return score

Grid Searching

In [17]:
gridsearch(cerebro_run, param_grid, scoring, datafeeds)

------
long () and short ()
Sell eth @ price: 294.25 for Qty: -13.848709996250978
Buy btc @ price: 4096.32 for Qty: 1
------
close position
Buy eth @ price: 296.88000000000005 for Qty: 13.848709996250978
Sell btc @ price: 4129.32 for Qty: -1
------
long () and short ()
Sell eth @ price: 299.1 for Qty: -13.661952524239384
Buy btc @ price: 4069.13 for Qty: 1
------
close position
Buy eth @ price: 332.12 for Qty: 13.661952524239384
Sell btc @ price: 3997.57 for Qty: -1
------
long () and short ()
Sell eth @ price: 309.96 for Qty: -11.936639029583507
Buy btc @ price: 3700.0 for Qty: 1
------
close position
Buy eth @ price: 311.16 for Qty: 11.936639029583507
Sell btc @ price: 3866.48 for Qty: -1
------
long () and short ()
Sell eth @ price: 307.08 for Qty: -12.76862706786505
Buy btc @ price: 3874.38 for Qty: 1
------
close position
Buy eth @ price: 318.48 for Qty: 12.76862706786505
Sell btc @ price: 4166.27 for Qty: -1
------
long () and short ()
Sell eth @ price: 323.42 for Qty: -13.344907

(0.8549533547222941, {'OPEN_THRE': 1, 'CLOS_THRE': 0.1, 'period': 30})